In [ ]:
!pip install -qU langchain sentence_transformers groq
!pip install -qU langchain-huggingface langchain-community
!pip install -qU faiss-cpu openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 10.3 MB/s eta 0:00:00


In [ ]:
from collections import Counter
from groq import Groq
from tqdm import tqdm
import numpy as np
import json
import faiss
import json
import pandas as pd
from openai import OpenAI
import os

from google.colab import drive, files

drive.mount('/content/drive', force_remount=True)

#API_KEY = ""
API_KEY = ""
#API_KEY = ""

client = Groq(
    api_key=API_KEY,
)

model_llm = "llama3-70b-8192"

os.environ["OPENAI_API_KEY"] = ""

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

Mounted at /content/drive


In [ ]:
csv_path = '/content/drive/MyDrive/IA024/ProjetoFinal/df/df_questions_answers.xlsx'

data_csv = pd.read_excel(csv_path)
data_csv.head()

,question_number,question_summary,question_text,answer,answer_cleaned,references,linked_questions
0,1,OBRIGATORIEDADE,Quem está obrigado a apresentar a Declaração d...,['Está obrigada a apresentar a Declaração de A...,['Está obrigada a apresentar a Declaração de A...,"Lei nº 9.250, de 26 de dezembro de 1995, art. ...",NaN
1,2,PESSOA FÍSICA DESOBRIGADA,Pessoa física desobrigada pode apresentar a De...,"['Sim. A pessoa física, ainda que desobrigada,...","['Sim. A pessoa física, ainda que desobrigada,...","Instrução Normativa RFB nº 2.178, de 5 de març...","['001', '174']"
2,3,TITULAR OU SÓCIO DE EMPRESA,Contribuinte que é titular ou sócio de empresa...,"['Não, a menos que se enquadre nas hipóteses p...","['Não, a menos que se enquadre nas hipóteses p...","Instrução Normativa RFB nº 2.178, de 5 de març...",['001']
3,4,QUADRO SOCIETÁRIO OU ASSOCIADO DE COOPERATIVA,"Contribuinte, que participou de quadro societá...","['Não, a menos que esteja obrigado a declarar ...","['Não, a menos que esteja obrigado a declarar ...","Instrução Normativa RFB nº 2.178, de 5 de març...",['001']
4,5,DE DECLARAR,Contribuinte que constou como responsável pera...,['Esse contribuinte está obrigado a declarar c...,['Esse contribuinte está obrigado a declarar c...,"Instrução Normativa RFB nº 2.178, de 5 de març...",['001']


In [ ]:
data_csv = data_csv.drop(columns=['question_summary', 'answer', 'references', 'references', 'linked_questions'])
data_csv.head()

,question_number,question_text,answer_cleaned
0,1,Quem está obrigado a apresentar a Declaração d...,['Está obrigada a apresentar a Declaração de A...
1,2,Pessoa física desobrigada pode apresentar a De...,"['Sim. A pessoa física, ainda que desobrigada,..."
2,3,Contribuinte que é titular ou sócio de empresa...,"['Não, a menos que se enquadre nas hipóteses p..."
3,4,"Contribuinte, que participou de quadro societá...","['Não, a menos que esteja obrigado a declarar ..."
4,5,Contribuinte que constou como responsável pera...,['Esse contribuinte está obrigado a declarar c...


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Carregando o tokenizer e o modelo Legal-BERT
tokenizer = AutoTokenizer.from_pretrained("stjiris/bert-large-portuguese-cased-legal-tsdae")
model = AutoModel.from_pretrained("stjiris/bert-large-portuguese-cased-legal-tsdae")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29794, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, 

In [ ]:
index = faiss.read_index("/content/drive/MyDrive/IA024/ProjetoFinal/faiss/faiss_index_v3.index")
index_acordaos = faiss.read_index("/content/drive/MyDrive/IA024/ProjetoFinal/faiss/faiss_index_acordaos_v3.index")

In [ ]:
json_acordaos_path = '/content/drive/MyDrive/IA024/ProjetoFinal/json/acordaos.json'

with open(json_acordaos_path, 'r') as file:
    acordaos_json = json.load(file)

acordaos = list(acordaos_json.values())

json_normas_path = '/content/drive/MyDrive/IA024/ProjetoFinal/json/normas_full.json'

with open(json_normas_path, 'r') as file:
    normas_json = json.load(file)

normas = list(normas_json.values())

In [ ]:
normas_acordaos_json = {**normas_json, **acordaos_json}

In [ ]:
list(acordaos_json.items())[0]

('10073720738201158_7133632.txt',
 'DOCUMENTO VALIDADO \nMINISTÉRIO DA FAZENDA \nConselho Administrativo de Recursos Fiscais \nPROCESSO  \n10073.720738/2011-58  \nRESOLUÇÃO \n1202-000.270 – 1ª SEÇÃO/2ª CÂMARA/2ª TURMA ORDINÁRIA    \nSESSÃO DE \n16 de julho de 2024 \nRECURSO \nVOLUNTÁRIO \nRECORRENTE \nUNIMED RESENDE RJ COOPERATIVA DE TRABALHO MEDICO \nINTERESSADO \nFAZENDA NACIONAL \nAssunto: Conversão do Julgamento em Diligência \nRESOLUÇÃO \nVistos, relatados e discutidos os presentes autos. \nResolvem os membros do colegiado, por unanimidade de votos, converter o \njulgamento do recurso em diligência. \nSala de Sessões, em 16 de julho de 2024. \n \nAssinado Digitalmente \nMaurício Novaes Ferreira – Relator \n \nAssinado Digitalmente \nLeonardo de Andrade Couto – Presidente \n \nParticiparam da sessão de julgamento os julgadores Mauricio Novaes Ferreira, \nAndre Luis Ulrich Pinto, Ana Cecilia Lustosa da Cruz (suplente convocada), Roney Sandro Freire \nCorrea, Miriam Costa Faccin (sup

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

delimiters = ["\nCAPÍTULO", "\nCapítulo", "\nSeção", "\nArt. ", "\n§",
              "\nI -", "\nII -", "\nIII -", "\nIV -", "\nV -", "\nVI -",
              "\nVII -", "\nVIII -", "\nIX -", "\nX -","\n"]

text_splitter = RecursiveCharacterTextSplitter(
    separators=delimiters,
    chunk_size=1000,
    chunk_overlap=100
)

chunks_normas = []
chunks_normas_acordaos = []
normas_acordaos = normas+acordaos

# Percorre cada item no JSON
for filename, doc in normas_json.items():
    # Remove a extensão ".txt" do nome do arquivo
    doc_name = filename.replace(".txt", "")

    # Divide o documento em chunks
    chunks = text_splitter.split_text(doc)

    # Adiciona o nome do documento a cada chunk
    for chunk in chunks:
        # Formata cada chunk com o nome do arquivo e o conteúdo
        chunk_with_name = f"{doc_name}: {chunk}"
        chunks_normas.append(chunk_with_name)

# Percorre cada item no JSON
for filename, doc in normas_acordaos_json.items():
    # Remove a extensão ".txt" do nome do arquivo
    doc_name = filename.replace(".txt", "")

    # Divide o documento em chunks
    chunks = text_splitter.split_text(doc)

    # Adiciona o nome do documento a cada chunk
    for chunk in chunks:
        # Formata cada chunk com o nome do arquivo e o conteúdo
        chunk_with_name = f"{doc_name}: {chunk}"
        chunks_normas_acordaos.append(chunk_with_name)

In [ ]:
import requests
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np


def gerar_embedding(texto):
    # Tokenizar o texto
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    # Pooling: Média dos embeddings de cada token (desconsidera o token [CLS] e [SEP])
    embeddings = outputs.last_hidden_state
    embeddings = embeddings.mean(dim=1)  # Média ao longo dos tokens
    return embeddings.numpy()

def gerar_resposta_rag(pergunta, index, df, embedding_model, k=3):
    # Gerar o embedding da pergunta usando o BERTimbau
    pergunta_embedding = gerar_embedding(pergunta)

    # Recuperar as passagens mais próximas
    _, indices = index.search(np.array(pergunta_embedding, dtype=np.float32), k)

    # TROCAR chunks_normas ou chunks_normas_acordaos
    df_chunks_normas = pd.DataFrame({"resposta": chunks_normas})
    # Obter os textos das passagens relevantes
    contextos = [df_chunks_normas.iloc[i]["resposta"] for i in indices[0]]
    contexto = " ".join(contextos)

    # Construir o prompt avançado
    prompt = f"""
      Você é um assistente virtual especializado em tributos brasileiros da Receita Federal. Responda à pergunta de forma completa e com base no contexto fornecido. Para garantir uma resposta precisa e fundamentada, siga estas etapas:

      Análise da Pergunta: Identifique o que a pergunta busca, considerando palavras-chave e aspectos específicos.

      Busca no Contexto: Examine o contexto em busca de trechos relevantes que se relacionem diretamente com a pergunta.

      Construção da Resposta:

      - Utilize as informações do contexto e organize a resposta de forma objetiva e precisa.
      - Cite diretamente as normas ou leis, utilizando as referências exatas sempre que possível, sem mencionar o termo "contexto", "referências", "fontes".
      - Avaliação de Suficiência: Se o contexto não for suficiente para responder de forma fundamentada, responda apenas: "Estou aprendendo e no futuro poderei responder a essa pergunta."

      Regras a seguir:

      - Não inclua frases de interação direta com o usuário.
      - Caso o contexto não seja suficiente, declare que a resposta será disponibilizada futuramente.
      - Se a resposta já tiver sido respondida, não procure mais informações para complementar em outros contextos.
      - Sempre cite a norma e artigo que foi retirada a informação ao final da resposta em formato de lista, para o usuário consultar, ela geralmente esta no começo dos contextos (serão enviados 3 em uma única string) antes dos ':' por exemplo: Lei nº 9.250, de 26 de dezembro de 1995, art. 25 ou Instrução Normativa RFB nº 2.178, de 5 de março de 2024, art. 2º.
      - Jamais cite parágrafos, incisos, §, alíneas e numerações, apenas normas e artigos nas respostas nem na lista.
      - Não mencione no meio da respostas nenhuma norma, artigo, parágrafos, incisos (§), alíneas e numerações, apenas na lista final de consulta do usuário.
      - Substitua RIR/2018 ou RIR2018 quando for citar ele, o nome real que deve ser citado é Decreto Nº 9.580.
      - Para o caso de acórdãos, a informação de qual é fica no contexto, não no nome e geralmente é assim: "Acórdão nº: 303-25.277 Nº do processo: 11065-001.472/87-94 Contribuinte: Bolivar & CIA", caso use algum em alguma resposta, cite-o na lista final também.
      - Não fale que consultou a normas ou artigos, apenas forneça a lista.

      Pergunta a ser respondida: "{pergunta}"

      Contexto: "{contexto}"
    """

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",
    )
    return chat_completion.choices[0].message.content, contextos

# Carregar o DataFrame com perguntas e respostas
# Certifique-se de que 'df' contém as colunas "pergunta" e "resposta"
df = data_csv[168:183]
#df = data_csv

# Adicionar uma nova coluna para salvar as respostas geradas
# MUDAR index ou index_acordaos
df[['resposta_rag', 'contexto']] = df["question_text"].apply(lambda pergunta: pd.Series(gerar_resposta_rag(pergunta, index, df, model)))

# Salvar o DataFrame atualizado com as respostas RAG

In [ ]:
df

,question_number,question_text,answer_cleaned,resposta_rag,contexto
0,1,Quem está obrigado a apresentar a Declaração d...,['Está obrigada a apresentar a Declaração de A...,A obrigação de apresentar a Declaração de Ajus...,[Lei nº 11.438: CAPÍTULO I\nDOS INCENTIVOS AO ...
1,2,Pessoa física desobrigada pode apresentar a De...,"['Sim. A pessoa física, ainda que desobrigada,...",Uma pessoa física desobrigada da apresentação ...,[Instrução Normativa RFB nº 1.704: Seção IV \n...
2,3,Contribuinte que é titular ou sócio de empresa...,"['Não, a menos que se enquadre nas hipóteses p...",O contribuinte que é titular ou sócio de uma e...,[Lei nº 11.438: CAPÍTULO I\nDOS INCENTIVOS AO ...
3,4,"Contribuinte, que participou de quadro societá...","['Não, a menos que esteja obrigado a declarar ...",A pergunta busca esclarecer as obrigações fisc...,[RIR2018: I - a pessoa jurídica resultante da ...
4,5,Contribuinte que constou como responsável pera...,['Esse contribuinte está obrigado a declarar c...,O contribuinte que constou como responsável pe...,[Instrução Normativa RFB nº 2.066: I - por pes...
...,...,...,...,...,...
572,711,Qual é o tratamento tributário dos rendimentos...,['Os rendimentos obtidos em conta de depósito ...,Os rendimentos produzidos por conta de depósit...,[Lei nº 7.713: Art. 44. O imposto de que trata...
573,712,Qual é o tratamento tributário dos rendimentos...,['Tratando-se de conta de depósito de poupança...,Os rendimentos produzidos por conta de depósit...,[Lei nº 9.430: § 4º Tratando-se de pessoa físi...
574,713,As importâncias correspondentes aos juros de c...,['Não. Relativamente aos juros de conta de dep...,As importâncias correspondentes aos juros de c...,[Lei nº 14.754: § 3º A variação cambial de dep...
575,714,Qual é o tratamento tributário aplicável na aq...,"['A entrega, pelo licitante vencedor, de títul...",O tratamento tributário aplicável na aquisição...,[RIR2018: § 3º O disposto neste artigo aplica-...


In [ ]:
df.to_excel("teste_total.xlsx", index = False)
files.download('teste_total.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>